In [9]:
from pathlib import Path

import darts
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from darts.dataprocessing.transformers import Scaler
from darts.models import NBEATSModel
from darts.timeseries import TimeSeries
from darts.utils.data import TrainingDataset
from darts.utils.missing_values import fill_missing_values
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Data Loading

In [7]:
def load_timeseries(
    file_path=Path("data", "LD2011_2014.txt"),
    freq="2H",
    start_time="2014-01-02 00:00:00",
    end_time="2014-09-01 00:00:00",
    components=[f"MT_{i:03}" for i in range(1, 51)],
) -> TimeSeries:
    df = pd.read_csv(file_path, sep=";", index_col=0, parse_dates=True, decimal=",")
    # df = df.replace(0, np.nan)

    df = df.loc[start_time:end_time, components]

    df = df.resample(freq).mean()

    ts = TimeSeries.from_dataframe(df, freq=freq)

    ts = ts.astype(np.float32)

    return ts


ts = load_timeseries()
# ts = fill_missing_values(ts, interpolate_kwargs=dict(method='time'))
assert not ts.pd_dataframe(copy=False).isna().any(axis=None)

In [8]:
df = ts.pd_dataframe(copy=False)

## Normalization

https://www.tensorflow.org/tutorials/structured_data/time_series#normalize_the_data

> It is important to scale features before training a neural network. Normalization is a common way of doing this scaling: subtract the mean and divide by the standard deviation of each feature.
>
> The mean and standard deviation should only be computed using the training data so that the models have no access to the values in the validation and test sets.
>
> It's also arguable that the model shouldn't have access to future values in the training set when training, and that this normalization should be done using moving averages. That's not the focus of this tutorial, and the validation and test sets ensure that you get (somewhat) honest metrics. So, in the interest of simplicity this tutorial uses a simple average.



In [5]:
# 70 - 10 - 10 split
ts_train_original, ts_nontrain_original = ts.split_before(0.8)
ts_val_original, ts_test_original = ts_nontrain_original.split_before(0.5)

In [6]:
# transformer = Scaler(MinMaxScaler())
transformer = Scaler(StandardScaler())
ts_train_transformed = transformer.fit_transform(ts_train_original)
ts_val_transformed = transformer.transform(ts_val_original)
ts_test_transformed = transformer.transform(ts_test_original)

In [ ]:
def optuna_objective(trial: optuna.Trial) -> float:
    pass

In [62]:
model = NBEATSModel(
    input_chunk_length=7 * 12,  # 7 days
    output_chunk_length=1 * 12,  # 1 day
    optimizer_kwargs=dict(
        lr=1e-3,
    ),
    pl_trainer_kwargs=dict(
        accelerator="gpu",
        devices=[0],
    ),
)

model.fit(
    series=ts_train_transformed,
    val_series=ts_val_transformed,
    epochs=10,
    num_loader_workers=2,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 39.1 M
---------------------------------------------------
39.1 M    Trainable params
26.5 K    Non-trainable params
39.1 M    Total params
156.504   Total

Epoch 9: 100%|██████████| 70/70 [00:18<00:00,  3.84it/s, train_loss=0.146, val_loss=1.240]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 70/70 [00:18<00:00,  3.84it/s, train_loss=0.146, val_loss=1.240]


NBEATSModel(generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=ReLU, input_chunk_length=84, output_chunk_length=12, optimizer_kwargs={'lr': 0.001}, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]})

In [66]:
preds = transformer.inverse_transform(
    model.predict(n=1 * 12, series=ts_test_transformed[: 7 * 12])
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 18.72it/s]


In [75]:
preds.end_time()

Timestamp('2014-08-15 16:00:00', freq='2H')

In [74]:
ts_test_original[7 * 12 : 8 * 12].end_time()

Timestamp('2014-08-15 16:00:00', freq='2H')